In [7]:
import kagglehub
import pandas as pd
import numpy as np
from transformers import TFBertModel, BertTokenizer
#import tensorflow as tf

In [8]:
# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /home/eugen/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [9]:
dataset = pd.read_csv(path + '/IMDB Dataset.csv')
pd.set_option('display.max_colwidth', None)
dataset.head(3), dataset.info(), dataset.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


(                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [10]:
dataset = dataset.sample(n=10000)
dataset['review'] = dataset['review'].apply(lambda row: row.lower())
#dataset['review'] = dataset['review'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
dataset.sample(3)

review  \
46583                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [11]:
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')

def tokenize_text(text):
    return tokenizer(
         text.tolist(),
         max_length = 200,
         truncation = True,
         padding = 'max_length',
         return_tensors = 'np'
    )

tokenized = tokenize_text(dataset['review'])

In [18]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

bert_model = TFBertModel.from_pretrained('prajjwal1/bert-tiny', from_pt=True)
outputs = bert_model(tokenized)

X = outputs.last_hidden_state[:, 0, :].numpy()
y = dataset['sentiment']


2025-06-28 18:13:10.784492: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleLoadData(&module, data)' failed with 'CUDA_ERROR_UNSUPPORTED_PTX_VERSION'

2025-06-28 18:13:10.784517: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleGetFunction(&function, module, kernel_name)' failed with 'CUDA_ERROR_INVALID_HANDLE'

2025-06-28 18:13:10.784524: W tensorflow/core/framework/op_kernel.cc:1844] INTERNAL: 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE'
2025-06-28 18:13:10.784536: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INTERNAL: 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE'


InternalError: Exception encountered when calling layer 'LayerNorm' (type LayerNormalization).

{{function_node __wrapped__SquaredDifference_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:SquaredDifference] name: 

Call arguments received by layer 'LayerNorm' (type LayerNormalization):
  • inputs=tf.Tensor(shape=(1, 2, 128), dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print(classification_report(y_test, y_pred))

NameError: name 'X' is not defined

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators=200, random_state=42)
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)
print(classification_report(y_test, y_pred_lgbm))

NameError: name 'X_train' is not defined